In [1]:
import os
os.chdir("..")
os.getcwd()

'/Users/laurasisson/odor-pair'

In [2]:
import analysis.best as best
model = best.get_model()
model

MixturePredictor(
  (gcn): GCN(
    (pad): ZeroPad2d((0, 91, 0, 0))
    (gcn): GINConv(nn=Sequential(
      (0): Sequential(
        (0): Linear(in_features=100, out_features=100, bias=True)
        (1): ReLU()
        (2): Dropout(p=0, inplace=False)
      )
      (1): Sequential(
        (0): Linear(in_features=100, out_features=100, bias=True)
        (1): ReLU()
        (2): Dropout(p=0, inplace=False)
      )
    ))
    (readout): Set2Set(100, 200)
    (post_mp): Sequential(
      (0): Sequential(
        (0): Linear(in_features=200, out_features=100, bias=True)
        (1): ReLU()
        (2): Dropout(p=0, inplace=False)
      )
      (1): Sequential(
        (0): Linear(in_features=100, out_features=100, bias=True)
      )
    )
  )
  (out): Sequential(
    (0): Sequential(
      (0): Linear(in_features=200, out_features=77, bias=True)
      (1): ReLU()
      (2): Dropout(p=0, inplace=False)
    )
    (1): Sequential(
      (0): Linear(in_features=77, out_features=77, bias=True)

In [3]:
import json
import os

folddir = "dataset/folds"

fold_fnames = os.listdir(folddir)
folds = []
for fname in fold_fnames:
    if not fname.endswith(".json"): 
        continue
    with open(os.path.join("dataset/folds",fname)) as f:
        folds.append(json.load(f))
folds[0].keys()

dict_keys(['train', 'test', 'validate', 'covered_notes'])

In [4]:
import pairing.data

def make_pairdatas(dataset,covered_notes,show_failed=False):
    pds = []
    failed = 0
    for d in dataset:
        try:
            notes = {n:n for n in d["blend_notes"]}
            pds.append(pairing.data.to_pairdata((d["edge"],notes),covered_notes))
        except AttributeError:
            failed+=1
    if show_failed:
        print(f"Built {len(pds)} datapoints. Failed on {failed}.")
    return pds

make_pairdatas(folds[0]["train"],folds[0]["covered_notes"])[0]

PairData(y=[49], x_s=[11, 9], edge_attr_s=[20, 3], edge_index_s=[2, 20], smiles_s='CC(=CCCC(C)(C=C)O)C', x_t=[18, 9], edge_attr_t=[36, 3], edge_index_t=[2, 36], smiles_t='C1CCCCCOCCCCOC(=O)CCCC1')

In [11]:
from tqdm.notebook import tqdm

fold_datas = []
for fold in tqdm(folds):
    covered = fold["covered_notes"]
    ftrain = make_pairdatas(fold["train"],covered)
    ftest = make_pairdatas(fold["test"],covered)
    fvalid = make_pairdatas(fold["validate"],covered)
    fold_datas.append({"train":ftrain,"test":ftest,"validate":fvalid,"covered_notes":covered})
len(fold_datas)

  0%|          | 0/5 [00:00<?, ?it/s]

5

In [38]:
from main import MixturePredictor, GCN

params = {"DIM":100,"LINEAR":2,"CONVS":6,"AGGR":2,"ARCH":"GIN","LR":0.0094574,"STEPS":124.00,"DECAY":0.0018544}
def do_trial(fold_idx,fold):
    model = MixturePredictor(num_convs=params["CONVS"],
        num_linear=params["LINEAR"],
        embedding_size=int(params["DIM"]),
        aggr_steps=params["AGGR"],
        architecture=params["ARCH"],
        num_classes=len(fold["covered_notes"]))

    bsz = int((2**18) / ((params["DIM"])*params["LINEAR"]))

    train, test, validate = fold["train"], fold["test"], fold["validate"]
    train_loader = pairing.data.loader(train,batch_size=bsz)
    test_loader = pairing.data.loader(test,batch_size=bsz)
    validate_loader = pairing.data.loader(validate,batch_size=bsz)

    loss_fn = torch.nn.BCEWithLogitsLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=params["LR"])
    # From paper
    end_step = .9 * params["STEPS"]
    scheduler = torch.optim.lr_scheduler.LinearLR(optimizer,
                                                  start_factor=1,
                                                  end_factor=params["DECAY"],
                                                  total_iters=end_step)
    def do_train_epoch():
        model.train()
        losses = []
        for batch in train_loader:
            optimizer.zero_grad()

            pred = model(**batch.to_dict())

            loss = loss_fn(pred,batch.y)
            loss.backward()
            losses.append(loss*len(batch.y))

            optimizer.step()

        return torch.stack(losses).sum() / len(train)

    def collate(loader):
        model.eval()
        preds = []
        ys = []
        for batch in loader:
            with torch.no_grad():
                pred = model(**batch.to_dict())

            preds.append(pred)
            ys.append(batch.y)

        return torch.cat(preds,dim=0), torch.cat(ys,dim=0)

    def get_loss(loader):
        pred, y = collate(loader)
        return loss_fn(pred,y)

    def get_auroc(loader):
        pred, y = collate(loader)
        return auroc(pred,y.int())

    for s in tqdm(range(int(params["STEPS"]))):
        train_loss = do_train_epoch()
        scheduler.step()
        validate_loss = get_loss(validate_loader)
        
    test_pred, test_true = collate(test_loader)
    torch.save(test_pred,f"fold_preds/{fold_idx}_pred.pt")
    torch.save(test_true,f"fold_preds/{fold_idx}_true.pt")
        
    return get_loss(test_loader), test_pred, test_true

test_preds = []
test_trues = []
for fidx, fold in enumerate(fold_datas):
    print(f"Fold {fidx}.")
    test_loss, test_pred, test_true = do_trial(fidx,fold)
    print(test_loss)
    test_preds.append(test_pred)
    test_trues.append(test_true)    

Fold 0.


  0%|          | 0/124 [00:00<?, ?it/s]

tensor(0.1025)
Fold 1.


  0%|          | 0/124 [00:00<?, ?it/s]

tensor(0.0910)
Fold 2.


...

tensor(0.1175)
Fold 3.


  0%|          | 0/124 [00:00<?, ?it/s]

tensor(0.1221)
Fold 4.


  0%|          | 0/124 [00:00<?, ?it/s]

tensor(0.1242)
